### Begin hier
Begin met het runnen van deze blokken code.  
Deze zijn verantwoordelijk voor imports, setup, dependencies en andere zaken. 

In [20]:
# Imports
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Dense, Input, Concatenate, Lambda
from keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib

import data_generator as dgen

print(device_lib.list_local_devices())
tf.debugging.set_log_device_placement(False)
#print(tf.__version__)
#print(len(tf.config.list_physical_devices('GPU'))>0)

#print("Succes loading Imports")

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13558652340354266355
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5738397696
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11565351071428127878
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [21]:
# Functions
def get_file_name(path):
    file_name = path[path.rfind('/') + 1:]
    # Splits out the file-type designation
    file_name, file_extension = file_name.split('.')
    return file_name
    
print("Succes loading Functions")

Succes loading Functions


### Heb je al data?
Als er al trainingsdata bestaat, schrijf dan simpelweg de filename bij het betreffende blok.  
Run anders het blok waarin de data nog word gegenereerd. 

In [4]:
"""Ik heb geen data"""
# Start here if no data has been generated.

print("Provide the path to a video file.\n"
      "  (Example: C:/Users/admin/video/video.mp4)")

vid_path = input("Provide a video path: ")
vid_name = get_file_name(vid_path)
print(f"Provided path: '{vid_path}',\nfound filename: '{vid_name}'")
print("")

generator = dgen.Generator(vid_path)
generator.generate_data()
print(f"Video has been processed and exported to {generator.file_name}__trainingsData.mp4")
print(f"Data generated and exported to {generator.file_name}_trainingsData.json")
print("")

try: 
      data_path = f"generateData_OUTPUT/{vid_name}_trainingsData.json"
      frame_data = pd.read_json(data_path, orient='index')
except Exception as e:
      print(f"Something went wrong.\n {e}")


print("The data-table was loaded. \nTotal Table size: ")
print(f"Rows: {frame_data.shape[0]}, Columns: {frame_data.shape[1]}")

Provide the path to a video file.
  (Example: C:/Users/admin/video/video.mp4)
Provided path: 'dataset bike/GOPR1126_Trim_360P.mp4',
found filename: 'GOPR1126_Trim_360P'

---========---
Intiated the Video Processor!
---========---


Downsampling is not required.
Current resolution: 360
Frame: 250
Frame: 500
Frame: 750
Frame: 1000
Frame: 1250
Frame: 1500
Frame: 1750
Frame: 2000
Frame: 2250
Frame: 2500
Frame: 2750
Frame: 3000
Frame: 3250
Frame: 3500
Frame: 3750
Frame: 4000
Frame: 4250
Frame: 4500
Frame: 4750
Frame: 5000
Frame: 5250
Frame: 5500
Frame: 5750
Frame: 6000
Frame: 6250
Frame: 6500
Frame: 6750
Frame: 7000
Frame: 7250
Frame: 7500
Frame: 7750
Frame: 8000
Frame: 8250
Frame: 8500
Frame: 8750
Frame: 9000
Frame: 9250
Frame: 9500
Frame: 9750
Frame: 10000
Frame: 10250
Frame: 10500
Frame: 10750
Frame: 11000
Frame: 11250
Frame: 11500
Frame: 11750
Frame: 12000
Frame: 12250
Frame: 12500
Frame: 12750
Frame: 13000
Frame: 13250
Frame: 13500
Frame: 13750
Frame: 14000
Frame: 14250
Frame: 14500
Fr

AttributeError: 'Generator' object has no attribute 'file_name'

In [22]:
"""Ik heb al trainingsdata. """
# Start here if data has already been generated.
print("Provide the filename to a file in "
      "the generateData_OUTPUT folder.\n"
      "Example: perfect_example_trainingsData")
      
data_path = input("Filename: ")
vid_name = get_file_name(data_path)
print(f"Provided path: {data_path}")

try: 
      data_path = f"generateData_OUTPUT/{vid_name}.json"
      frame_data = pd.read_json(data_path, orient='index')
except Exception as e:
      print(f"Something went wrong.\n {e}")

print("\nTotal Table size: ")
print(f"Rows: {frame_data.shape[0]}, Columns: {frame_data.shape[1]}")

Provide the filename to a file in the generateData_OUTPUT folder.
Example: perfect_example_trainingsData
Provided path: generateData_OUTPUT/GOPR1125_Trim_360P_trainingsData.json

Total Table size: 
Rows: 19543, Columns: 482


In [23]:
"""preprocessing data"""
#Splitting data into X and Y
Xtemp = frame_data.drop(columns=['TS', 'HR', 'PWR', 'heartRateZone'])
Y = frame_data["heartRateZone"]
X = []
temp = []


#Extracting values in lists in lists
for index, row in Xtemp.iterrows():
    for i in range(0,478):
        for g in range(0,3):
            temp.append(row[i][0][g])
    X.append(temp)
    temp = []
Y.to_numpy()
Y = Y.values.reshape(-1,1)
temps = np.zeros((len(Y), 6))


#encoding the y data
for i in range(0,len(Y)):
    if Y[i][0] == 0:
        temps[i][0] = 1
    elif Y[i][0] == 1:
        temps[i][1] = 1
    elif Y[i][0] == 2:
        temps[i][2] = 1
    elif Y[i][0] == 3:
        temps[i][3] = 1
    elif Y[i][0] == 4:
        temps[i][4] = 1
    elif Y[i][0] == 5:
        temps[i][5] = 1

#split the data into a train and test dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, temps, test_size=0.2, random_state=42)

X_train = np.array(X_train)
X_test = np.array(X_test)

# print(X_train.shape)
# print(Y_train.shape)

In [ ]:
# to clear the backend
#keras.backend.clear_session()

In [25]:
#input shape include all the x y z coordinates from the facemesh points
inputTensor = Input(shape=(1434,))
layers = []

#making a first layer that links the x y and z coordinate from a points to a single neuron
for i in range(0, 1434, 3):
    layers.append(Lambda(lambda x: x[:,i: i+3], output_shape=((3,)))(inputTensor))
for i in range(0, 478):
    layers[i] = Dense(1)(layers[i])
coordslayer = Concatenate()(layers)

#Hidden layers
hidden1 = Dense(477*2, activation='relu')(coordslayer)
hidden2 = Dense(477, activation='relu')(hidden1)
hidden3 = Dense(240, activation='relu')(hidden2)
hidden4 = Dense(100, activation='relu')(hidden3)
hidden5 = Dense(20, activation='relu')(hidden4)

#output layer that includes the encoded heartrate data
outputTensor = Dense(6, activation='sigmoid')(hidden5)
model = Model(inputs=inputTensor, outputs=outputTensor)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 1434)]       0           []                               
                                                                                                  
 lambda_478 (Lambda)            (None, 3)            0           ['input_2[0][0]']                
                                                                                                  
 lambda_479 (Lambda)            (None, 3)            0           ['input_2[0][0]']                
                                                                                                  
 lambda_480 (Lambda)            (None, 3)            0           ['input_2[0][0]']                
                                                                                            

In [17]:
# Train the keras model
model.fit(X_train, Y_train, epochs=150, batch_size=1000)


Epoch 1/150
14/14 [==============================] - 2s 123ms/step - loss: 2.5434 - accuracy: 0.3186
Epoch 2/150
14/14 [==============================] - 1s 100ms/step - loss: 1.3738 - accuracy: 0.4575
Epoch 3/150
14/14 [==============================] - 1s 99ms/step - loss: 1.2250 - accuracy: 0.5081
Epoch 4/150
14/14 [==============================] - 1s 98ms/step - loss: 1.1782 - accuracy: 0.5214
Epoch 5/150
14/14 [==============================] - 1s 97ms/step - loss: 1.1558 - accuracy: 0.5326
Epoch 6/150
14/14 [==============================] - 1s 98ms/step - loss: 1.1402 - accuracy: 0.5398
Epoch 7/150
14/14 [==============================] - 1s 98ms/step - loss: 1.1378 - accuracy: 0.5400
Epoch 8/150
14/14 [==============================] - 1s 98ms/step - loss: 1.1235 - accuracy: 0.5479
Epoch 9/150
14/14 [==============================] - 1s 97ms/step - loss: 1.1137 - accuracy: 0.5483
Epoch 10/150
14/14 [==============================] - 1s 96ms/step - loss: 1.1114 - accuracy: 0.55

In [27]:
# evaluate the keras model
accuracy = model.evaluate(X_test, Y_test)

123/123 [==============================] - 9s 45ms/step - loss: 1.0467 - accuracy: 0.5705


In [19]:
#model saved
model.save('models/model3 faces')
print("model Saved")

INFO:tensorflow:Assets written to: models/model3 faces\assets


In [26]:
#model Loaded
model = keras.models.load_model('models/model3 faces')
print("model Loaded")